<a href="https://colab.research.google.com/github/ahcamachod/1893-word2vec-interpretacion-del-lenguaje-humano-con-word-embedding/blob/aula-3/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec: Interpretación del lenguaje humano con Word Embedding

En este notebook desarrollaremos un clasificador de noticias para una start-up de noticias llamada **Alura Latam News**.

El modelo Word2Vec utilizado para entrenar nuestro clasificador fue tomado de la siguiente fuente: 
 
Aitor Almeida, Aritz Bilbao Jayo. (2018) "Word2vec models for the Spanish Language." Available from: https://github.com/aitoralmeida/spanish_word2vec

## 1. Explorando el Dataset 

In [1]:
import pandas as pd

entrenamiento = pd.read_csv("/content/drive/MyDrive/word2vec/noticias_entrenamiento.csv")
test = pd.read_csv("/content/drive/MyDrive/word2vec/noticias_prueba.csv")
entrenamiento.sample(5)

,fecha,titulo,pais,extracto,resumen,enlace,categoria
86741,2022-03-18 11:15:00,Occidente pone en riesgo su seguridad al armar...,PR,"El embajador ruso ante la ONU, Vasili Nebenzia...","El embajador ruso ante la ONU, Vasili Nebenzia...",https://www.noticel.com/mundo/top-stories/2022...,mundo
65498,2022-03-20 17:41:00,Rodríguez destacó el potencial de la producció...,AR,"El ministro de Desarrollo Agrario bonaerense, ...","El ministro de Desarrollo Agrario bonaerense, ...",https://www.eldestapeweb.com/economia/buenos-a...,economia
55145,2022-03-31 14:58:11,"""El Gobierno puede garantizar que no va a falt...",AR,"La portavoz presidencial indicó que este 2022,...","La portavoz de la Presidencia, Gabriela Cerrut...",https://www.baenegocios.com/negocios/El-Gobier...,economia
27460,2022-04-04 22:30:00,"El rosado, la tendencia 2022 que Eiza González...",AR,"Si te tienes que comprar una prenda, entonces ...",Eiza González está siempre al salto de las ten...,https://www.mdzol.com/mdz-femme/moda/2022/4/4/...,ciencia y tecnologia
13367,2022-03-29 11:02:40,Lorenzo Milá presenta la nueva etapa de 'En po...,ES,El veterano espacio se emitirá los miércoles e...,'En portada' inicia una nueva etapa con varios...,https://www.elperiodico.com/es/yotele/20220329...,entretenimiento


In [2]:
test.sample(5)

,fecha,titulo,pais,extracto,resumen,enlace,categoria
21495,2022-03-18 23:24:12,¿Cómo será la inauguración del AIFA? Sedena ex...,GT,"El titular de la Sedena, Luis Cresencio Sandov...",Este 21 de marzo será la inauguración del Aero...,https://www.tvazteca.com/aztecanoticias/notas/...,politica
8806,2022-03-31 21:59:17,Aida Victoria critica a Marbelle por comentari...,CO,El trino 'racista' de Marbelle a Francia Márqu...,"El martes 29 de marzo, la cantante Maureen Bel...",https://www.rcnradio.com/entretenimiento/aida-...,entretenimiento
4784,2022-04-05 00:35:04,Video: Olivia Rodrigo tira su Grammy y lo rompe,US,La cantante Olivia Rodrigo ganó tres gramófono...,La cantante Olivia Rodrigo ganó tres gramófono...,https://www.dallasnews.com/espanol/al-dia/espe...,entretenimiento
667,2022-04-03 23:14:07,Lista de ganadores de los premios Grammy,US,Lista parcial de ganadores de los premios Gram...,Lista parcial de ganadores de los premios Gram...,https://www.milfordmirror.com/entertainment/ar...,entretenimiento
4386,2022-04-03 14:00:29,Ciencia en Panamá: Ciencia a bordo de mini sub...,PA,Los vehículos sumergibles han permitido a los ...,Por Helkin Guevara (Corresponsal digital)\n\n\...,https://www.tvn-2.com/contenido_exclusivo/Cien...,entretenimiento


In [3]:
entrenamiento.shape

(91844, 7)

In [4]:
test.shape

(22961, 7)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vector = CountVectorizer()

In [6]:
texto = [
         "tenga un buen día",
         "tenga un excelente día",
         "tenga un pésimo día luna sol carro casa familia"
]

In [7]:
vector.fit(texto)

CountVectorizer()

In [8]:
vector.vocabulary_

{'buen': 0,
 'carro': 1,
 'casa': 2,
 'día': 3,
 'excelente': 4,
 'familia': 5,
 'luna': 6,
 'pésimo': 7,
 'sol': 8,
 'tenga': 9,
 'un': 10}

In [9]:
vector_bueno = vector.transform(['buen'])

In [10]:
vector_luna = vector.transform(['luna'])

In [11]:
print(vector_bueno)

  (0, 0)	1


In [12]:
vector_bueno

<1x11 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [13]:
print(vector_bueno.toarray())

[[1 0 0 0 0 0 0 0 0 0 0]]


In [14]:
print(vector_luna.toarray())

[[0 0 0 0 0 0 1 0 0 0 0]]


## 2. Cargando un modelo Word2Vec

In [15]:
!unzip '/content/drive/MyDrive/word2vec/keyed_vectors.zip'

Archive:  /content/drive/MyDrive/word2vec/keyed_vectors.zip
  inflating: complete.kv             
  inflating: complete.kv.vectors.npy  


In [16]:
from gensim.models import KeyedVectors

modelo = KeyedVectors.load('/content/complete.kv')

## 3. Entendiendo Word2Vec

## 4. Combinando Vectores

## 5. Clasificando textos